In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from openai import api_key
from sympy.physics.units import temperature
%pip install ragas



In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = ""

In [ ]:
%pip install datasets

In [ ]:
from datasets import load_dataset

dataset_macro = load_dataset("json", data_files="")
dataset_old = load_dataset("json", data_files="")
raw_dataset_macro = dataset_macro['train']
raw_dataset_old = dataset_old['train']


In [ ]:
%pip install --upgrade ragas


In [ ]:
print(len(raw_dataset_macro["question"][0]))
print(raw_dataset_macro["answer"][0][len(raw_dataset_macro["question"])-1-0])

In [ ]:
from datasets import Dataset
data = []
for i in range(len(raw_dataset_macro["question"][0])):
    data.append({
        "user_input": raw_dataset_macro["question"][0][i],
        "retrieved_contexts": raw_dataset_macro["contexts"][0][i],  # 假设是个列表或字符串
        "response": raw_dataset_macro["answer"][0][len(raw_dataset_macro["question"][0])-1-i],
        "reference": raw_dataset_macro["reference"][0][i]
    })
data_dict_macro = data

In [ ]:
from datasets import Dataset
data = []
for i in range(len(raw_dataset_old["question"][0])):
    data.append({
        "user_input": raw_dataset_old["question"][0][i],
        "retrieved_contexts": raw_dataset_old["contexts"][0][i],  # 假设是个列表或字符串
        "response": raw_dataset_old["answer"][0][len(raw_dataset_old["question"][0]) - 1 - i],
        "reference": raw_dataset_old["reference"][0][i]
    })
data_dict_old = data

In [ ]:
from ragas import EvaluationDataset
import pandas as pd

processed_dataset_macro = EvaluationDataset.from_list(data_dict_macro)
processed_dataset_old = EvaluationDataset.from_list(data_dict_old)

In [ ]:
from openai import OpenAI
from ragas.embeddings import OpenAIEmbeddings
import os
# 获取 embeddings
openai_client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_API_BASE"]
)

embeddings = OpenAIEmbeddings(client=openai_client, model="text-embedding-3-large")


In [ ]:
import ragas
print(ragas.__version__)

In [ ]:
from ragas import evaluate
from ragas.llms import llm_factory
from ragas.run_config import RunConfig
import os

from ragas.metrics import (
    Faithfulness,
    FactualCorrectness,
    ContextRelevance, ResponseRelevancy,
)


metrics = [
    ResponseRelevancy(strictness=3),
    Faithfulness(),
    FactualCorrectness(),
    ContextRelevance(),
]

llm = llm_factory(
    model="gpt-4o-mini",
    base_url=os.environ["OPENAI_API_BASE"],
)
llm.multiple_completion_supported = True

for m in metrics:
    m.llm = llm
    m.embeddings = embeddings

In [ ]:
from ragas import RunConfig

result_macro = evaluate(
    dataset=processed_dataset_macro,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings,
    batch_size=20,
    raise_exceptions=False,
)
evaluation_macro = result_macro.to_pandas()


In [ ]:
result_old = evaluate(
    dataset=processed_dataset_old,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings,
    batch_size=20,
    raise_exceptions=False,
)
evaluation_old = result_old.to_pandas()

In [ ]:
evaluation_macro.to_json("output_macro_response_relevancy_reference_as_answer_wrong.json", orient="records", indent=4)
evaluation_old.to_json("output_old_response_relevancy_reference_as_answer_wrong.json", orient="records", indent=4)